In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
from typing import Callable
import pandas as pd

In [2]:
a = np.array([[1, 0, 2], [2, 2, 1]])
a.cumsum(axis=1)

a > 0

array([[ True, False,  True],
       [ True,  True,  True]])

In [3]:
def SPRT(a, b, mu) -> (float, float, float, float):
    A = b / (1 - a)
    B = (1 - b) / a
    
    #First order
    X = ss.norm.rvs(loc = 0, scale = 1, size=(10000, 1000))
    Y = ss.norm.logpdf(X, loc = 0, scale = 1) - ss.norm.logpdf(X, loc = mu, scale = 1)
    Z = Y.cumsum(axis=1)
    T = -(np.log(A) <= Z).astype(int) * (np.log(B) >= Z).astype(int)
    E_0 = T.mean()
    alp = np.sum(T > np.log(B))
    
    #Second order
    X = ss.norm.rvs(loc = mu, scale = 1, size=(10000, 1000))
    Y = ss.norm.logpdf(X, loc = 0, scale = 1) - ss.norm.logpdf(X, loc = mu, scale = 1)
    Z = Y.cumsum(axis=1)
    
    T0 = np.min(np.where(np.log(A) > Z), axis = 1)
    T1 = np.min(np.where(np.log(B) < Z), axis = 1)
    T = min(T0, T1)
    E_1 = T.mean()
    bet = np.sum(T < np.log(A))
    
    return alp, bet, E_0, E_1
    

#  Домашка.

In [4]:
df = pd.read_csv('hw_6_data.txt', sep=' ', names=['first', 'second', 'third', 'winning', 'reporter?'])
df.head()

,first,second,third,winning,reporter?
0,2,0,0,0,1
1,0,5,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [5]:
report = df[df['reporter?'] == 1]
report.head()

,first,second,third,winning,reporter?
0,2,0,0,0,1
1,0,5,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [6]:
report = report - (report > 4)
report.head()

,first,second,third,winning,reporter?
0,2,0,0,0,1
1,0,4,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [10]:
expected = np.array([1 / 2, 1 / 4, 1 / 8, 1 / 16, 1 /64, 1 / 64, 1 / 32])

names = ['first', 'second', 'third']
observed = np.unique(report[names], return_counts=True)[1]
observed

ss.chisquare(observed, expected * 138 * 3)

Power_divergenceResult(statistic=6.178743961352657, pvalue=0.4034680378126573)

In [30]:
f = ss.expon().rvs(size=20, random_state=1337)

In [27]:
stat = ss.ks_1samp(f, ss.expon().cdf).statistic
stat

0.14068311278543333

In [29]:
BIG = 100000
gf = ss.expon().rvs(size=(BIG, 20), random_state=1337)
np.sum(np.apply_along_axis(lambda a: ss.ks_1samp(a, ss.expon(loc = 0).cdf).statistic, 1, gf) > stat) / BIG

0.77235